In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/')

In [3]:
col_types = {
    'FIPS_Code': str,
    'District': str,
    'CD':str
}

CD_df = pd.read_csv('data/House_Reps_District.csv', dtype=col_types)

some ZIP Codes, such as "unique" ZIPs that are often tied to an internal mail-routing mechanism (such as UC Berkeley which has its own internal ZIP code and mail-routing system) will have very low ratios). 

In [4]:
ZIP_to_CD_df = pd.read_excel('Data/Partisanship and Health Behavior/Data/ZIP_CD_062020.xlsx')

ZIP_to_CD_df = ZIP_to_CD_df.rename(columns={
    'RES_RATIO': 'CD_RES_RATIO',
    'BUS_RATIO': 'CD_BUS_RATIO',
    'OTH_RATIO': 'CD_OTH_RATIO',
    'TOT_RATIO': 'CD_TOT_RATIO'
})

ZIP_to_CD_df['ZIP'] = ZIP_to_CD_df['ZIP'].astype(str).str.zfill(5)

In [5]:
col_types = {
    'DELIVERY ZIPCODE': str
}

FULL_ZIPS = pd.read_excel('Data/Partisanship and Health Behavior/Data/Full_Zips.xlsx', dtype=col_types)

FULL_ZIPS = FULL_ZIPS[['DELIVERY ZIPCODE','PHYSICAL STATE']]

FULL_ZIPS = FULL_ZIPS.rename(columns={
    'DELIVERY ZIPCODE': 'ZIP'
})

FULL_ZIPS.head()

/Users/chrissoria/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,ZIP,PHYSICAL STATE
0,00601,PR
1,00602,PR
2,00603,PR
3,00604,PR
4,00605,PR


For now, I will assign a ZIP code to whatever whatever County it falls mostly under residentially \
This will NOT include DC simply because DC doesn't have a CD \
IN A FUTURE VERSION, I WILL HAVE TO START WITH A LIST OF ZIP CODES THAT INCLUDE DISTRICT OF COLUMBIA'S (WHICH I  HAVE ABOVE)

In [6]:
col_types = {
    'COUNTY': str,
    'ZIP': str
}

ZIP_to_County_df = pd.read_excel('Data/Partisanship and Health Behavior/Data/ZIP_COUNTY_062020_HUD.xlsx', dtype=col_types)
ZIP_to_County_df['COUNTY'] = ZIP_to_County_df['COUNTY'].astype(str).str.zfill(5)
ZIP_to_County_df['ZIP'] = ZIP_to_County_df['ZIP'].astype(str).str.zfill(5)

ZIP_to_County_df = pd.merge(FULL_ZIPS, ZIP_to_County_df, how ='outer', on = "ZIP")

# Sort the DataFrame by RES_RATIO in descending order
ZIP_to_County_df = ZIP_to_County_df.sort_values(by='RES_RATIO', ascending=False)
ZIP_to_County_df = ZIP_to_County_df.drop_duplicates(subset='ZIP', keep='first')

ZIP_to_County_df.reset_index(drop=True, inplace=True)

ZIP_to_County_df = ZIP_to_County_df.rename(columns={
    'RES_RATIO': 'COUNTY_RES_RATIO',
    'BUS_RATIO': 'COUNTY_BUS_RATIO',
    'OTH_RATIO': 'COUNTY_OTH_RATIO',
    'TOT_RATIO': 'COUNTY_TOT_RATIO'
})

In [7]:
ZIP_FEATURES = pd.merge(ZIP_to_CD_df, ZIP_to_County_df, on = "ZIP", how = "left")

ZIP_FEATURES

,ZIP,CD,CD_RES_RATIO,CD_BUS_RATIO,CD_OTH_RATIO,CD_TOT_RATIO,PHYSICAL STATE,COUNTY,COUNTY_RES_RATIO,COUNTY_BUS_RATIO,COUNTY_OTH_RATIO,COUNTY_TOT_RATIO
0,00501,3601,0.0,1.0,0.0,1.0,NY,36103,0.000000,1.000000,0.000000,1.000000
1,00601,7200,1.0,1.0,1.0,1.0,PR,72001,0.839242,0.800983,0.871166,0.837603
2,00602,7200,1.0,1.0,1.0,1.0,PR,72003,1.000000,0.998801,1.000000,0.999919
3,00603,7200,1.0,1.0,1.0,1.0,PR,72005,0.997152,0.998160,1.000000,0.997290
4,00604,7200,1.0,1.0,1.0,1.0,PR,72005,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
45909,99925,0200,0.0,0.0,1.0,1.0,AK,02198,0.000000,0.000000,1.000000,1.000000
45910,99926,0200,0.0,0.0,1.0,1.0,AK,02198,0.000000,0.000000,1.000000,1.000000
45911,99927,0200,0.0,0.0,1.0,1.0,AK,02198,0.000000,0.000000,1.000000,1.000000
45912,99928,0200,0.0,0.0,1.0,1.0,AK,02130,0.000000,0.000000,1.000000,1.000000


Maybe it doesn't make sense to have ZIP to CD and ZIP to County in the same dataset. I might make these into two later on, but for now this contains the county under which the ZIP code mostly falls for residential addresses. 

In [8]:
ZIP_FEATURES['CD'] = ZIP_FEATURES['CD'].astype(str)
CD_df['CD'] = CD_df['CD'].astype(str)


ZIP_FEATURES = pd.merge(ZIP_FEATURES, CD_df, on = "CD", how= "left")
ZIP_FEATURES = ZIP_FEATURES.drop(columns=['FIPS_Code','Years_Served','District','CD_RES_RATIO',
                                                      'CD_BUS_RATIO','CD_OTH_RATIO','COUNTY_RES_RATIO',
                                                      'COUNTY_BUS_RATIO','COUNTY_OTH_RATIO'])

ZIP_FEATURES = ZIP_FEATURES.rename(columns={
    'COUNTY': 'COUNTY_FIPS',
    'Name': 'CONGRESSPERSON',
    'State': 'STATE',
    'Party': 'CONGRESSPERSON_PARTY'
})

cols_to_move = ZIP_FEATURES.columns[3:5]
df_remaining = ZIP_FEATURES.drop(columns=cols_to_move)
ZIP_FEATURES = pd.concat([df_remaining, ZIP_FEATURES[cols_to_move]], axis=1)

ZIP_FEATURES['STATE'] = ZIP_FEATURES.apply(
    lambda row: row['PHYSICAL STATE'] if pd.isna(row['STATE']) and not pd.isna(row['PHYSICAL STATE']) else row['STATE'], 
    axis=1
)

ZIP_FEATURES = ZIP_FEATURES.drop(columns='PHYSICAL STATE')

ZIP_FEATURES

,ZIP,CD,CD_TOT_RATIO,COUNTY_TOT_RATIO,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,COUNTY_FIPS
0,00501,3601,1.0,1.000000,"Zeldin, Lee M.",New York,Republican,36103
1,00601,7200,1.0,0.837603,"González-Colón, Jenniffer",Puerto Rico,Republican,72001
2,00602,7200,1.0,0.999919,"González-Colón, Jenniffer",Puerto Rico,Republican,72003
3,00603,7200,1.0,0.997290,"González-Colón, Jenniffer",Puerto Rico,Republican,72005
4,00604,7200,1.0,1.000000,"González-Colón, Jenniffer",Puerto Rico,Republican,72005
...,...,...,...,...,...,...,...,...
47479,99925,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198
47480,99926,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198
47481,99927,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198
47482,99928,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02130


Pulling from their API to see if I can extract any more years worth of data \
It looks like it's only cumulative.

In [9]:
import requests
import json

# API endpoint
url = "https://data.cdc.gov/resource/kn79-hsxy.json"

# Making a GET request
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    
    # Extracting all 'data_as_of' values
    data_as_of_values = [item['data_as_of'] for item in data if 'data_as_of' in item]

    # Getting unique values by converting the list to a set
    unique_data_as_of_values = set(data_as_of_values)

    # Printing unique 'data_as_of' values
    print("Unique 'data_as_of' values:")
    for value in unique_data_as_of_values:
        print(value)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

Unique 'data_as_of' values:
2023-06-28T00:00:00.000


Below, we need to add a leading "0" to the FIPS code (FIPS codes are all 5 digits) \
I'll borrw Urban Rural Code from here

In [10]:
col_types = {
    'FIPS County Code': str
}

CDC_deaths = pd.read_csv('Data/Partisanship and Health Behavior/Data/Provisional_COVID-19_Death_Counts_in_the_United_States_by_County.csv', dtype=col_types)
CDC_deaths['FIPS County Code'] = CDC_deaths['FIPS County Code'].astype(str).str.zfill(5)

CDC_deaths = CDC_deaths.drop(columns = ['Date as of','Deaths from All Causes','Footnote','State'])
CDC_deaths.head()

,Start Date,End Date,County name,FIPS County Code,Urban Rural Code,Deaths involving COVID-19
0,01/01/2020,06/24/2023,Aleutians East Borough,02013,Noncore,NaN
1,01/01/2020,06/24/2023,Anchorage Municipality,02020,Medium metro,775.0
2,01/01/2020,06/24/2023,Bethel Census Area,02050,Noncore,40.0
3,01/01/2020,06/24/2023,Denali Borough,02068,Noncore,NaN
4,01/01/2020,06/24/2023,Dillingham Census Area,02070,Noncore,NaN


In [11]:
col_types = {
    'countyFIPS': str
}

usafacts_deaths = pd.read_csv('Data/Partisanship and Health Behavior/Data/covid_deaths_usafacts.csv', dtype=col_types)
usafacts_deaths['countyFIPS'] = usafacts_deaths['countyFIPS'].astype(str).str.zfill(5)

For this preliminary analysis, I will only use wave 2,4,6. \
Each wave's data was collected on 2020-06-17 - 2020-06-23, 2020-11-29 - 2020-12-16, 2021-05-12 - 2021-05-25. \
I will take covid mortality from the midpoint of the wave range. \
Wave 2 = 6/20/2020 \
Wave 4 = 12/9/2020 \
Wave 6 = 5/19/2021

In [12]:
columns_to_keep = ['countyFIPS', 'County Name', '2020-06-20', '2020-12-09', '2021-05-19']

usafacts_deaths = usafacts_deaths[columns_to_keep]

usafacts_deaths = usafacts_deaths.rename(columns={
    'countyFIPS': 'COUNTY_FIPS',
    '2020-06-20': 'COUNTY_COV_DEATHS_2020_06_20',
    '2020-12-09': 'COUNTY_COV_DEATHS_2020_12_09',
    '2021-05-19': 'COUNTY_COV_DEATHS_2021_05_19'
})

In [13]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, usafacts_deaths, on = "COUNTY_FIPS", how = "left")
ZIP_FEATURES

,ZIP,CD,CD_TOT_RATIO,COUNTY_TOT_RATIO,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,COUNTY_FIPS,County Name,COUNTY_COV_DEATHS_2020_06_20,COUNTY_COV_DEATHS_2020_12_09,COUNTY_COV_DEATHS_2021_05_19
0,00501,3601,1.0,1.000000,"Zeldin, Lee M.",New York,Republican,36103,Suffolk County,1963.0,2093.0,3383.0
1,00601,7200,1.0,0.837603,"González-Colón, Jenniffer",Puerto Rico,Republican,72001,NaN,NaN,NaN,NaN
2,00602,7200,1.0,0.999919,"González-Colón, Jenniffer",Puerto Rico,Republican,72003,NaN,NaN,NaN,NaN
3,00603,7200,1.0,0.997290,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,NaN,NaN,NaN,NaN
4,00604,7200,1.0,1.000000,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
47479,99925,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198,Prince of Wales - Hyder Census Area,0.0,1.0,1.0
47480,99926,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198,Prince of Wales - Hyder Census Area,0.0,1.0,1.0
47481,99927,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198,Prince of Wales - Hyder Census Area,0.0,1.0,1.0
47482,99928,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02130,Ketchikan Gateway Borough,0.0,0.0,1.0


In [14]:
col_types1 = {
    'STATE': str,
    'COUNTY': str
}

col_types2 = {
    'County Code (FIPS)': str,
    'County Subdivision Code (FIPS)': str,
    'Place Code (FIPS)': str,
    'Consolidtated City Code (FIPS)': str,
    'State Code (FIPS)': str
}


census_pop = pd.read_csv('Data/Partisanship and Health Behavior/Data/co-est2021-alldata.csv', encoding='ISO-8859-1',
                        dtype = col_types1)
census_pop_FIPS = pd.read_excel('Data/Partisanship and Health Behavior/Data/all-geocodes-v2020.xlsx', 
                                dtype = col_types2)

census_pop_FIPS = census_pop_FIPS.rename(columns={
    'Area Name (including legal/statistical area description)': 'CTYNAME'
})

census_pop['COUNTY_FIPS'] = census_pop['STATE']+census_pop['COUNTY']

columns_to_keep = ['STATE','POPESTIMATE2020','DEATHS2020','POPESTIMATE2021','DEATHS2021','COUNTY_FIPS']

census_pop = census_pop[columns_to_keep]

census_pop = census_pop.rename(columns={
    'STATE': 'STATE_FIPS'
})

census_pop

,STATE_FIPS,POPESTIMATE2020,DEATHS2020,POPESTIMATE2021,DEATHS2021,COUNTY_FIPS
0,01,5024803,16148,5039877,64868,01000
1,01,58877,168,59095,681,01001
2,01,233140,661,239294,2867,01003
3,01,25180,109,24964,394,01005
4,01,22223,90,22477,282,01007
...,...,...,...,...,...,...
3189,56,42158,116,41614,400,56037
3190,56,23347,29,23575,96,56039
3191,56,20441,49,20635,166,56041
3192,56,7658,32,7705,115,56043


A crude mortality rate, not standardized or age adjusted, for this preliminary analysis \
I manually insert Puerto Rico' State FIPS code and District of Columbia to make this a clean merge across the board.

In [15]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, census_pop, on = "COUNTY_FIPS", how = "left")


ZIP_FEATURES['COV_County_CMR_2020_06_20'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2020_06_20'] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
ZIP_FEATURES['COV_County_CMR_2020_12_09'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2020_12_09'] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
ZIP_FEATURES['COV_County_CMR_2021_05_19'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2021_05_19'] / ZIP_FEATURES['POPESTIMATE2021']) * 1000
ZIP_FEATURES['STATE_FIPS'] = ZIP_FEATURES.apply(lambda row: 72 if row['STATE'] == "Puerto Rico" else row['STATE_FIPS'], axis=1)

ZIP_FEATURES

,ZIP,CD,CD_TOT_RATIO,COUNTY_TOT_RATIO,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,COUNTY_FIPS,County Name,COUNTY_COV_DEATHS_2020_06_20,COUNTY_COV_DEATHS_2020_12_09,COUNTY_COV_DEATHS_2021_05_19,STATE_FIPS,POPESTIMATE2020,DEATHS2020,POPESTIMATE2021,DEATHS2021,COV_County_CMR_2020_06_20,COV_County_CMR_2020_12_09,COV_County_CMR_2021_05_19
0,00501,3601,1.0,1.000000,"Zeldin, Lee M.",New York,Republican,36103,Suffolk County,1963.0,2093.0,3383.0,36,1524099.0,3806.0,1526344.0,15459.0,1.287974,1.373270,2.216407
1,00601,7200,1.0,0.837603,"González-Colón, Jenniffer",Puerto Rico,Republican,72001,NaN,NaN,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00602,7200,1.0,0.999919,"González-Colón, Jenniffer",Puerto Rico,Republican,72003,NaN,NaN,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00603,7200,1.0,0.997290,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,NaN,NaN,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00604,7200,1.0,1.000000,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,NaN,NaN,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47479,99925,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198,Prince of Wales - Hyder Census Area,0.0,1.0,1.0,02,5725.0,26.0,5729.0,36.0,0.000000,0.174672,0.174551
47480,99926,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198,Prince of Wales - Hyder Census Area,0.0,1.0,1.0,02,5725.0,26.0,5729.0,36.0,0.000000,0.174672,0.174551
47481,99927,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198,Prince of Wales - Hyder Census Area,0.0,1.0,1.0,02,5725.0,26.0,5729.0,36.0,0.000000,0.174672,0.174551
47482,99928,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02130,Ketchikan Gateway Borough,0.0,0.0,1.0,02,13902.0,26.0,13754.0,106.0,0.000000,0.000000,0.072706


In [16]:
State_FIPS_Crosswalk = ZIP_FEATURES[['STATE','STATE_FIPS']]
State_FIPS_Crosswalk['STATE_FIPS'] = State_FIPS_Crosswalk['STATE_FIPS'].astype(str)
State_FIPS_Crosswalk = State_FIPS_Crosswalk.drop_duplicates(subset='STATE', keep='first')
State_FIPS_Crosswalk = State_FIPS_Crosswalk[pd.notna(State_FIPS_Crosswalk['STATE'])].reset_index(drop=True)

State_FIPS_Crosswalk.to_csv('data/STATE_FIPS_CROSSWALK.csv', index = False)
State_FIPS_Crosswalk.to_csv('data/Partisanship and Health Behavior/Data/STATE_FIPS_CROSSWALK.csv', index = False)

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_75193/2987085650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  State_FIPS_Crosswalk['STATE_FIPS'] = State_FIPS_Crosswalk['STATE_FIPS'].astype(str)


In [17]:
columns_to_keep = ['FIPS','RUCC_2013']

col_types = {
    'FIPS': str,
}

RUC_county_df = pd.read_csv('Data/Partisanship and Health Behavior/Data/ruralurbancodes2013.csv', dtype = col_types)

RUC_county_df = RUC_county_df[columns_to_keep]

RUC_county_df = RUC_county_df.rename(columns={
    'RUCC_2013': 'COUNTY_RUCC_2013',
    'FIPS':'COUNTY_FIPS'
})

RUC_county_df['COUNTY_FIPS'] = RUC_county_df['COUNTY_FIPS'].astype(str)

ZIP_FEATURES = pd.merge(ZIP_FEATURES, RUC_county_df, on = "COUNTY_FIPS", how = "left")

Next, I want to control for the severity of the lockdowns according to Oxford's tracker

In [18]:
State_Restrictions_df = pd.read_csv('Data/Partisanship and Health Behavior/Data/OxCGRT_USA_differentiated_withnotes_2020.csv')
State_Restrictions_df['Date'] = State_Restrictions_df['Date'].astype(str)
State_Restrictions_df = State_Restrictions_df[State_Restrictions_df['Date'].isin(['20200617', '20201129'])]
columns_to_keep = ['RegionName','GovernmentResponseIndex_SimpleAverage','Date']
State_Restrictions_df = State_Restrictions_df[columns_to_keep]

State_Restrictions_df = State_Restrictions_df.rename(columns={
    'RegionName': 'STATE',
    'GovernmentResponseIndex_SimpleAverage':'State_Government_Response_Index'
})

State_Restrictions_df

/Users/chrissoria/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (82,93,121,122) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,STATE,State_Government_Response_Index,Date
168,NaN,62.50,20200617
333,NaN,67.19,20201129
534,Alaska,57.81,20200617
699,Alaska,53.65,20201129
900,Alabama,59.64,20200617
...,...,...,...
18267,Wisconsin,53.65,20201129
18468,West Virginia,61.46,20200617
18633,West Virginia,56.25,20201129
18834,Wyoming,59.90,20200617


In [19]:
State_Restrictions_df['State_Government_Response_Index_06172020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['Date'] == "20200617" else "0", axis=1
)

State_Restrictions_df['State_Government_Response_Index_06172020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['State_Government_Response_Index_06172020'] == "0" else row['State_Government_Response_Index_06172020'], axis=1
)

State_Restrictions_df['State_Government_Response_Index_11192020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['Date'] == "20201129" else "0", axis=1
)

State_Restrictions_df['State_Government_Response_Index_11192020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['State_Government_Response_Index_11192020'] == "0" else row['State_Government_Response_Index_11192020'], axis=1
)

State_Restrictions_df

,STATE,State_Government_Response_Index,Date,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020
168,NaN,62.50,20200617,62.50,62.50
333,NaN,67.19,20201129,67.19,67.19
534,Alaska,57.81,20200617,57.81,57.81
699,Alaska,53.65,20201129,53.65,53.65
900,Alabama,59.64,20200617,59.64,59.64
...,...,...,...,...,...
18267,Wisconsin,53.65,20201129,53.65,53.65
18468,West Virginia,61.46,20200617,61.46,61.46
18633,West Virginia,56.25,20201129,56.25,56.25
18834,Wyoming,59.90,20200617,59.90,59.90


In [20]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, State_Restrictions_df, on = "STATE", how = "left")

ZIP_FEATURES

,ZIP,CD,CD_TOT_RATIO,COUNTY_TOT_RATIO,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,COUNTY_FIPS,County Name,COUNTY_COV_DEATHS_2020_06_20,...,POPESTIMATE2021,DEATHS2021,COV_County_CMR_2020_06_20,COV_County_CMR_2020_12_09,COV_County_CMR_2021_05_19,COUNTY_RUCC_2013,State_Government_Response_Index,Date,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020
0,00501,3601,1.0,1.000000,"Zeldin, Lee M.",New York,Republican,36103,Suffolk County,1963.0,...,1526344.0,15459.0,1.287974,1.373270,2.216407,1.0,76.30,20200617,76.30,76.30
1,00501,3601,1.0,1.000000,"Zeldin, Lee M.",New York,Republican,36103,Suffolk County,1963.0,...,1526344.0,15459.0,1.287974,1.373270,2.216407,1.0,75.78,20201129,75.78,75.78
2,00601,7200,1.0,0.837603,"González-Colón, Jenniffer",Puerto Rico,Republican,72001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN
3,00602,7200,1.0,0.999919,"González-Colón, Jenniffer",Puerto Rico,Republican,72003,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
4,00603,7200,1.0,0.997290,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92980,99927,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198,Prince of Wales - Hyder Census Area,0.0,...,5729.0,36.0,0.000000,0.174672,0.174551,9.0,53.65,20201129,53.65,53.65
92981,99928,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02130,Ketchikan Gateway Borough,0.0,...,13754.0,106.0,0.000000,0.000000,0.072706,7.0,57.81,20200617,57.81,57.81
92982,99928,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02130,Ketchikan Gateway Borough,0.0,...,13754.0,106.0,0.000000,0.000000,0.072706,7.0,53.65,20201129,53.65,53.65
92983,99929,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02275,City and Borough of Wrangell,0.0,...,2055.0,67.0,0.000000,0.000000,0.000000,9.0,57.81,20200617,57.81,57.81


Next, I want to read in the county level presidential data and generate two variables \
1. For which president did the county overall vote for?
2. What percentage of the county voted for which president?

note: I manually added in DC because it was not in the dataset

In [21]:
col_types = {
    'county_fips': str,
}
county_pres = pd.read_csv('Data/Partisanship and Health Behavior/Data/countypres_2000-2020.csv', dtype = col_types)
county_pres = county_pres[county_pres['year'] == 2020].reset_index(drop=True)

county_pres['party_fips'] = county_pres['party'].astype(str) + county_pres['county_fips'].astype(str)

#there was an issue where there were multiple types of votes on each line for the same person
total_votes_by_party = county_pres.groupby('party_fips')['candidatevotes'].sum().reset_index()
total_votes_by_party.rename(columns={'candidatevotes': 'total_candidatevotes'}, inplace=True)

county_pres = county_pres.merge(total_votes_by_party, on='party_fips', how='left')

county_pres['percentage_won'] = (county_pres['total_candidatevotes'] / county_pres['totalvotes'])*100

# Group by 'countyfips' and find the index of the row with the largest 'percentage_won' in each group
county_pres_wide = county_pres.groupby('county_fips')['percentage_won'].idxmax()

# Select rows from the original DataFrame corresponding to these indices
county_pres_wide = county_pres.loc[county_pres_wide]

county_pres_wide = county_pres_wide.rename(columns={
    'candidate': 'winning_pres_candidate',
    'party':'winning_pres_party',
    'percentage_won': 'winning_pres_candidate_percentage'
})

#merge in the dems
dem_county_pres = county_pres[county_pres['party'] == 'DEMOCRAT']

dem_county_pres.rename(columns={'percentage_won': 'biden_percentage_won'}, inplace=True)

dem_county_pres = dem_county_pres[['biden_percentage_won','county_fips']]

county_pres_wide = county_pres_wide.merge(dem_county_pres, on='county_fips', how='left')

#merge in the reps
rep_county_pres = county_pres[county_pres['party'] == 'REPUBLICAN']

rep_county_pres.rename(columns={'percentage_won': 'trump_percentage_won'}, inplace=True)

rep_county_pres = rep_county_pres[['trump_percentage_won','county_fips']]

county_pres_wide = county_pres_wide.merge(rep_county_pres, on='county_fips', how='left')

#keeping only the columns I want
county_pres_wide = county_pres_wide.drop(columns =['year','state','state_po','county_name','office','candidatevotes','totalvotes','version','mode','party_fips'])

county_pres_wide

/Users/chrissoria/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,county_fips,winning_pres_candidate,winning_pres_party,total_candidatevotes,winning_pres_candidate_percentage,biden_percentage_won,trump_percentage_won
0,01001,DONALD J TRUMP,REPUBLICAN,19838,71.436802,27.018365,71.436802
1,01003,DONALD J TRUMP,REPUBLICAN,83544,76.171373,22.409030,76.171373
2,01005,DONALD J TRUMP,REPUBLICAN,5622,53.451226,45.788173,53.451226
3,01007,DONALD J TRUMP,REPUBLICAN,7525,78.426264,20.698280,78.426264
4,01009,DONALD J TRUMP,REPUBLICAN,24711,89.571553,9.569378,89.571553
...,...,...,...,...,...,...,...
12865,56037,DONALD J TRUMP,REPUBLICAN,12229,73.236316,22.894957,73.236316
12866,56039,JOSEPH R BIDEN JR,DEMOCRAT,9848,66.599040,66.599040,29.356868
12867,56041,DONALD J TRUMP,REPUBLICAN,7496,79.247278,16.819960,79.247278
12868,56043,DONALD J TRUMP,REPUBLICAN,3245,80.481151,16.145833,80.481151


In [22]:
county_pres_wide = county_pres_wide.rename(columns={
    'county_fips': 'COUNTY_FIPS'
})

ZIP_FEATURES = pd.merge(ZIP_FEATURES, county_pres_wide, on = "COUNTY_FIPS", how = "left")

ZIP_FEATURES

,ZIP,CD,CD_TOT_RATIO,COUNTY_TOT_RATIO,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,COUNTY_FIPS,County Name,COUNTY_COV_DEATHS_2020_06_20,...,State_Government_Response_Index,Date,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020,winning_pres_candidate,winning_pres_party,total_candidatevotes,winning_pres_candidate_percentage,biden_percentage_won,trump_percentage_won
0,00501,3601,1.0,1.000000,"Zeldin, Lee M.",New York,Republican,36103,Suffolk County,1963.0,...,76.30,20200617,76.30,76.30,DONALD J TRUMP,REPUBLICAN,375821.0,49.588654,48.556692,49.588654
1,00501,3601,1.0,1.000000,"Zeldin, Lee M.",New York,Republican,36103,Suffolk County,1963.0,...,75.78,20201129,75.78,75.78,DONALD J TRUMP,REPUBLICAN,375821.0,49.588654,48.556692,49.588654
2,00601,7200,1.0,0.837603,"González-Colón, Jenniffer",Puerto Rico,Republican,72001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00602,7200,1.0,0.999919,"González-Colón, Jenniffer",Puerto Rico,Republican,72003,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00603,7200,1.0,0.997290,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335629,99927,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198,Prince of Wales - Hyder Census Area,0.0,...,53.65,20201129,53.65,53.65,NaN,NaN,NaN,NaN,NaN,NaN
335630,99928,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02130,Ketchikan Gateway Borough,0.0,...,57.81,20200617,57.81,57.81,NaN,NaN,NaN,NaN,NaN,NaN
335631,99928,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02130,Ketchikan Gateway Borough,0.0,...,53.65,20201129,53.65,53.65,NaN,NaN,NaN,NaN,NaN,NaN
335632,99929,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02275,City and Borough of Wrangell,0.0,...,57.81,20200617,57.81,57.81,NaN,NaN,NaN,NaN,NaN,NaN


Next, for the sake of consistency, I want to get data on the percentage of the CD that was won by a democrat and Republican, respectively. 

In [23]:
col_types = {
    'DISTRICT': str,
}

county_CD_share = pd.read_excel('Data/Partisanship and Health Behavior/Data/federalelections2020.xlsx', 
                                sheet_name='13_US_House_Results_State', 
                                dtype=col_types)

county_CD_share = county_CD_share.rename(columns={
    'DISTRICT': 'CD_DISTRICT'
})

county_CD_share = county_CD_share[pd.notna(county_CD_share['GENERAL %'])].reset_index(drop=True)

county_CD_share = pd.merge(county_CD_share, State_FIPS_Crosswalk, how='left', on="STATE")

county_CD_share['CD'] = county_CD_share['STATE_FIPS'] + county_CD_share['CD_DISTRICT']

D_county_CD_share = county_CD_share[county_CD_share['PARTY'] == "D"].reset_index(drop=True)
D_county_CD_share = D_county_CD_share[['GENERAL %','CD']]

D_county_CD_share = D_county_CD_share.rename(columns={
    'GENERAL %': 'CD_PERCENT_DEMOCRAT'
})

R_county_CD_share = county_CD_share[county_CD_share['PARTY'] == "R"].reset_index(drop=True)
R_county_CD_share = R_county_CD_share[['GENERAL %','CD']]

R_county_CD_share = R_county_CD_share.rename(columns={
    'GENERAL %': 'CD_PERCENT_REPUBLICAN'
})

ZIP_FEATURES = pd.merge(ZIP_FEATURES, D_county_CD_share, how = 'left', on = "CD")
ZIP_FEATURES = pd.merge(ZIP_FEATURES, R_county_CD_share, how = 'left', on = "CD")

ZIP_FEATURES

,ZIP,CD,CD_TOT_RATIO,COUNTY_TOT_RATIO,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,COUNTY_FIPS,County Name,COUNTY_COV_DEATHS_2020_06_20,...,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020,winning_pres_candidate,winning_pres_party,total_candidatevotes,winning_pres_candidate_percentage,biden_percentage_won,trump_percentage_won,CD_PERCENT_DEMOCRAT,CD_PERCENT_REPUBLICAN
0,00501,3601,1.0,1.000000,"Zeldin, Lee M.",New York,Republican,36103,Suffolk County,1963.0,...,76.30,76.30,DONALD J TRUMP,REPUBLICAN,375821.0,49.588654,48.556692,49.588654,0.429142,0.482131
1,00501,3601,1.0,1.000000,"Zeldin, Lee M.",New York,Republican,36103,Suffolk County,1963.0,...,75.78,75.78,DONALD J TRUMP,REPUBLICAN,375821.0,49.588654,48.556692,49.588654,0.429142,0.482131
2,00601,7200,1.0,0.837603,"González-Colón, Jenniffer",Puerto Rico,Republican,72001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00602,7200,1.0,0.999919,"González-Colón, Jenniffer",Puerto Rico,Republican,72003,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00603,7200,1.0,0.997290,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533357,99927,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02198,Prince of Wales - Hyder Census Area,0.0,...,53.65,53.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544012
533358,99928,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02130,Ketchikan Gateway Borough,0.0,...,57.81,57.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544012
533359,99928,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02130,Ketchikan Gateway Borough,0.0,...,53.65,53.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544012
533360,99929,0200,1.0,1.000000,"Young, Don",Alaska,Republican,02275,City and Borough of Wrangell,0.0,...,57.81,57.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544012


In [24]:
ZIP_FEATURES.to_csv('data/ZIP_Features.csv', index = False)

In [25]:
ZIP_FEATURES.to_csv('data/Partisanship and Health Behavior/Data/ZIP_Features.csv', index = False)

Why is resp_educ and education different?